In [1]:
import sys
sys.path.append('../')

In [7]:
from env import DataSource, TradeBroker, TradingEnv

In [8]:
d = DataSource("../../Data/DAT_MT_EURUSD_M1_2021.csv", end_index=30)
tb = TradeBroker(d, initial_balance=10, price_history_length=5)
env = TradingEnv(tb)

In [9]:
env.reset()

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [10]:
for i in range(40):
    obs, reward, done, info = env.step(2)
    if done is True: 
        break

2024-03-08 00:18:44,414 - env - DEBUG - Holding at 1.22322
2024-03-08 00:18:44,417 - env - DEBUG - Holding at 1.22315
2024-03-08 00:18:44,420 - env - DEBUG - Holding at 1.22356
2024-03-08 00:18:44,424 - env - DEBUG - Holding at 1.22341
2024-03-08 00:18:44,427 - env - DEBUG - Holding at 1.2237
2024-03-08 00:18:44,430 - env - DEBUG - Holding at 1.2236
2024-03-08 00:18:44,432 - env - DEBUG - Holding at 1.22354
2024-03-08 00:18:44,434 - env - DEBUG - Holding at 1.22339
2024-03-08 00:18:44,438 - env - DEBUG - Holding at 1.22343
2024-03-08 00:18:44,440 - env - DEBUG - Holding at 1.22333
2024-03-08 00:18:44,442 - env - DEBUG - Holding at 1.22333
2024-03-08 00:18:44,444 - env - DEBUG - Holding at 1.22333
2024-03-08 00:18:44,446 - env - DEBUG - Holding at 1.22366
2024-03-08 00:18:44,448 - env - DEBUG - Holding at 1.22392
2024-03-08 00:18:44,451 - env - DEBUG - Holding at 1.2236
2024-03-08 00:18:44,453 - env - DEBUG - Holding at 1.22359
2024-03-08 00:18:44,455 - env - DEBUG - Holding at 1.22398


In [11]:
env.ste

2024-03-08 00:18:49,913 - env - DEBUG - No more data to return


TypeError: cannot unpack non-iterable NoneType object